# Thruster experiment Overview

In [ ]:
# %load imports.py
# %load ../imports.py
%matplotlib inline
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False  ## (To fix autocomplete)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import seaborn as sns
width=20
height=3
plt.rcParams["figure.figsize"] = (width,height)
sns.set(rc={'figure.figsize':(width,height)})

#import seaborn as sns
import os
from collections import OrderedDict

from IPython.display import display

pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)

import folium
import plotly.express as px
import plotly.graph_objects as go

import sys
import os

from sklearn.metrics import r2_score

import scipy.integrate
import seaborn as sns

import pyarrow as pa
import pyarrow.parquet as pq

import statsmodels.api as sm

from d2e2f.visualization import visualize
import scipy.integrate

from scipy.stats import norm
#from myst_nb import glue
#plt.style.use('presentation')
from IPython.display import display, Markdown, Latex
import sympy as sp
from d2e2f.reporting import pop_index, reload_kedro
import yaml
catalog = reload_kedro()

In [ ]:
#df_stat = catalog.load('uraniborg_experiment.trip_statistics_clean')
df_stat = catalog.load('uraniborg_experiment.steaming.trip_statistics')
df_stat['start_time'] = pd.to_datetime(df_stat['start_time'], utc=True)
df_stat['end_time'] = pd.to_datetime(df_stat['end_time'], utc=True);
df_stat['start_time'] = df_stat['start_time'].apply(lambda x : x.tz_convert(tz='Europe/Berlin'))
df_stat['end_time'] = df_stat['end_time'].apply(lambda x : x.tz_convert(tz='Europe/Berlin'))
#df_stat.describe()

The port manoeuvres were ex

Statistics for steaming data (excluding in port manoeuvres) between the points in {numref}`fig:steaming`.

```{figure} steaming.png
---
name: fig:steaming
---
The trips are cut at longitudes indicated by the black lines. 
```


![](steaming.png)

In [ ]:
df = catalog.load('uraniborg_experiment.data_steaming')
visualize.plot_trips(df)

In [ ]:
df_clean = df_stat.copy()

Time history of fuel consumption for all trips during the experiment.

In [ ]:
def extract(df_clean, start, end):
    start_index = (df_clean['start_time'] - start).abs().idxmin()
    end_index = (df_clean['start_time'] - end).abs().idxmin()
    df=df_clean.loc[start_index:end_index].copy()
    steps = df_clean['trip_no'] - df_clean.loc[start_index]['trip_no']
    super_trips = np.floor(steps / 2)
    df['operation'] = super_trips.apply(lambda x : 'ÖST' if x % 2 == 0 else 'BFH')
    return df

In [ ]:
start_experiment = pd.to_datetime("2022-08-19 14:15:00+2", utc=False).tz_convert(tz='Europe/Berlin')
end_experiment = pd.to_datetime("2022-08-22 06:50:00+2", utc=False).tz_convert(tz='Europe/Berlin')
start_pre_experiment = pd.to_datetime("2022-07-08 14:15:00+2", utc=False).tz_convert(tz='Europe/Berlin')
end_pre_experiment = pd.to_datetime("2022-07-11 06:50:00+2", utc=False).tz_convert(tz='Europe/Berlin')

df_experiment = extract(df_clean, start=start_experiment, end=end_experiment)
df_experiment['group'] = 'experiment'
df_pre_experiment = extract(df_clean, start=start_pre_experiment, end=end_pre_experiment)
df_pre_experiment['group'] = 'pre experiment'
df_data = df_pre_experiment.append(df_experiment)

In [ ]:
fig,ax=plt.subplots()
fig.set_size_inches(20,5)
facegrid = sns.scatterplot(data=df_data, x='start_time',y='consumption', hue='group', ax=ax);
ax.set_ylabel('(Fuel consumption per trip) [l]');

fig = ax.get_figure()
fig_name = 'overview'

In [ ]:
sns.displot(df_data, x='consumption',hue='group', kind="kde");

In [ ]:
interesting = ['sog','PR','consumption']
means = df_data.groupby(by=['group'], sort=False).mean()
means[interesting]

In [ ]:
means[['consumption']].transpose().plot.bar()

In [ ]:
time_table = catalog.load("uraniborg_experiment.time_table")
time_table.index = time_table.index.tz_convert(tz='Europe/Berlin')
assert (df_data['operation'] == time_table['operation'].values).all()
assert (df_data['trip_direction'] == time_table['direction'].values).all()

Uraniborg is run with the following time table during the experiment. The operation of the ship is switching every other roundtrip Landskrona-Ven-Landskrona between the captain (BFH) and by the chief mate (ÖST).

In [ ]:
time_table.head()

In [ ]:
df_data['time_table'] = time_table.index.tz_convert(tz='Europe/Berlin')

In [ ]:
df_data['delay'] = (df_data['start_time']) - df_data['time_table']

In [ ]:
assert (df_data['delay'] < pd.to_timedelta("10T")).all()

The trips operated by ÖST and BFH are shown below.

In [ ]:
px.scatter(data_frame=df_data, x='start_time', y='consumption', color='operation')

In [ ]:
df_experiment.groupby(by='operation').count()['consumption']

In [ ]:
df_experiment.groupby(by='operation')['consumption'].mean()

In [ ]:
df_experiment.groupby(by='operation')['consumption'].sum()/22

In [ ]:
df_data.groupby(by=['operation','group'], sort=False)['consumption'].sum()/22

In [ ]:
df_data.groupby(by=['operation','group'], sort=False)['consumption'].mean()

In [ ]:
sums = df_data.groupby(by=['operation','group'], sort=False)['consumption'].sum()
sum_experiment = sums.loc[('BFH','experiment')]
(sums-sum_experiment)/sum_experiment

In [ ]:
means = df_data.groupby(by=['operation','group'], sort=True).mean()
mean_BFH_experiment = means.loc[('BFH','experiment')]
comparison = (means-mean_BFH_experiment)/mean_BFH_experiment

In [ ]:
comparison

In [ ]:
means[['sog','PR','consumption']]

In [ ]:
comparison[['sog','PR','consumption']]

In [ ]:
z1 = df_data.groupby(by=['operation','group'], sort=False).get_group(('ÖST','experiment',))['consumption']
z2 = df_data.groupby(by=['operation','group'], sort=False).get_group(('BFH','experiment',))['consumption']

In [ ]:
from scipy.stats import ttest_ind
stat, p = ttest_ind(z1, z2)
if p > 0.05:
    print('probably the same distribution')
else:
    print('probably not the same distribution')

In [ ]:
stat, p = ttest_ind(z1-np.mean(z1), z2-np.mean(z2))
if p > 0.05:
    print('probably the same distribution')
else:
    print('probably not the same distribution')

In [ ]:
from statsmodels.stats import weightstats as stests
diff = 0.14*45.96
ztest ,pvalue = stests.ztest(z1, z2, value=diff)

if pvalue > 0.05:
    print('difference is significant')
else:
    print('difference is not significant')


In [ ]:
alpha = 0.95
rejected = pvalue < 1-alpha
if rejected:
    display(Markdown(f"Null hypotesis is rejected since p-value: {np.round(pvalue,3)} is smaller than {np.round(1-alpha,3)} ({round(alpha*100,0)}% confidence)"))
else:
    display(Markdown(f"Null hypotesis cannot be rejected since p-value: {np.round(pvalue,3)} is larger than {np.round(1-alpha,3)} ({round(alpha*100,0)}% confidence)"))

The consumption is plotted vs. the power ratio (PR) in the figure below. PR=1 means that only the aft thruster is run.

In [ ]:
df_experiment = df_data.groupby('group').get_group('experiment')
df_pre_experiment = df_data.groupby('group').get_group('pre experiment')

In [ ]:
df_data['case'] = df_data.apply(lambda df : f"{df['operation']} {df['group']}", axis=1)
px.scatter(data_frame=df_data, x='PR', y='consumption', color='case', hover_data=["time_table","sog"])

There is however a difference in the average speed over ground (sog) between ÖST and BFH as seen in the figure below.

In [ ]:
px.scatter(data_frame=df_data, x='PR', y='sog', color='case', hover_data=["time_table"])

In [ ]:
means = df_data.groupby(by=['operation','group'], sort=False).mean()

fig,ax=plt.subplots()
means['sog'].plot.bar(ax=ax)
ax.set_ylim(4,5);

To make a more fair comparison, some trips are excluded so that the mean of the average trip speed for ÖST and BFH is the same:

In [ ]:
mask_BFH_pre_experiment = ((df_data['operation'] == 'BFH') & (df_data['group'] == 'pre experiment') & (df_data['sog'] > 3.5) & (df_data['sog'] < 10.0))
mask_BFH_experiment = ((df_data['operation'] == 'BFH') & (df_data['group'] == 'experiment') & (df_data['sog'] > 4.5) & (df_data['sog'] < 10.0) & (df_data['PR'] > 0.9))
mask_ÖST_pre_experiment = ((df_data['operation'] == 'ÖST') & (df_data['group'] == 'pre experiment') & (df_data['sog'] > 3.0) & (df_data['sog'] < 5.18))
mask_ÖST_experiment = ((df_data['operation'] == 'ÖST') & (df_data['group'] == 'experiment') & (df_data['sog'] > 3.0) & (df_data['sog'] < 4.9))

df_data_select = pd.concat([df_data.loc[mask_BFH_pre_experiment],
                            df_data.loc[mask_BFH_experiment],
                            df_data.loc[mask_ÖST_pre_experiment],
                            df_data.loc[mask_ÖST_experiment],
                           ])
                            

df_data['speed classify'] = 'excluded'
df_data.loc[mask_experiment, 'speed classify'] = 'experiment'
df_data.loc[mask_others, 'speed classify'] = 'others'

In [ ]:
means = df_data_select.groupby(by=['operation','group'], sort=False).mean()

fig,ax=plt.subplots()
means['sog'].plot.bar(ax=ax)
ax.set_ylim(4,5);

In [ ]:
px.scatter(data_frame=df_data_select, x='PR', y='sog', color='case', hover_data=["time_table"])

In [ ]:
px.scatter(data_frame=df_data_select, x='PR', y='consumption', color='case', hover_data=["time_table"])

In [ ]:
len(df_data)

In [ ]:
len(df_data_select)

In [ ]:
means = df_data_select.groupby(by=['operation','group'], sort=True).mean()
mean_BFH_experiment = means.loc[('BFH','experiment')]
comparison = (means-mean_BFH_experiment)/mean_BFH_experiment

comparison[['sog','PR','consumption']]

In [ ]:
df_data_select.groupby(by=['operation','group'], sort=False)['sog'].count()

In [ ]:
df_data.plot(x='sog',y='consumption', style='.')

In [ ]:
df_data.head()

In [ ]:
y = df_data['consumption']

def features(df):
    X = pd.DataFrame(index=df.index)
    #X['sog'] = df['sog']
    X['sog**2'] = df['sog']**2
    #X['sog**3'] = df['sog**3']
    
    X['PR'] = df['PR']
    X['intercept'] = 1
    return X
X=features(df_data)
model = sm.OLS(y,X,hasconst=False)
result = model.fit()
result.summary()

In [ ]:
sogs = np.linspace(df_data['sog'].min(), df_data['sog'].max(),10)
PRs = np.linspace(df_data['PR'].min(), df_data['PR'].max(),5)

S,P = np.meshgrid(sogs, PRs)
df_ = pd.DataFrame()
df_['sog'] = S.flatten()
df_['PR'] = P.flatten()
X_ = features(df_)
df_['consumption'] = result.predict(X_)

In [ ]:
fig,ax=plt.subplots()
df_data.plot(x='sog',y='consumption',style='o', ax=ax)

for PR, group in df_.groupby(by='PR'):
    
    group.plot(x='sog', y='consumption', label=f'{PR}', ax=ax)


In [ ]:
fig,ax=plt.subplots()
ax.plot(y,result.predict(X),'.')
ax.plot([y.min(),y.max()], [y.min(),y.max()], 'r')
ax.set_aspect('equal', 'box')

In [ ]:
df_corrected = df_data.copy()
df_corrected['sog'] = df_data['sog'].mean()
X_corrected = features(df_corrected)
X = features(df_data)
y = df_data['consumption']
model_error = y - result.predict(X)

In [ ]:
df_corrected['consumption'] = result.predict(X_corrected) + model_error

In [ ]:
fig,ax=plt.subplots()

df_data.plot(x='sog', y='consumption', ax=ax, style='g.')
ax.plot(df_data['sog'], df_corrected['consumption'],'r.')
ax.plot([df_data['sog'].mean(),df_data['sog'].mean()],[20,80],'b--')

dx=df_corrected['sog'] - df_data['sog']
dy=df_corrected['consumption'] - df_data['consumption']

for (_,data),(_,corrected) in zip(df_data.iterrows(),df_corrected.iterrows()):
    ax.plot([data['sog'],data['sog']], [data['consumption'],corrected['consumption']], 'k-', zorder=-10)


In [ ]:
means = df_corrected.groupby(by=['operation','group'], sort=True).mean()
mean_BFH_experiment = means.loc[('BFH','experiment')]
comparison = (means-mean_BFH_experiment)/mean_BFH_experiment

comparison[['sog','PR','consumption']]

In [ ]:
means[['consumption']]

## Conclusion

BFH, which is utlizing more of the aft thruster, has **14%** lower fuel consumption than ÖST. If this improvement originate entirely from the changed thrust allocation is not certain. Perhaps BFH is also a more skilled operator than ÖST from the start? Trips prior to this experiment made by BFH and ÖST will be further investigated to also see if this is the case. Regardless of this the present result show that it should be possible to reduce the fuel consumption by around 20%, also accounting for other improvments when comparing the operation of BFH and ÖST, such as speed and speed profile etc.